In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib as plt
import glob
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd

![image.png](attachment:8db0485b-e771-471f-833b-e93f2da2efd5.png)

# LIMPIEZA

In [ ]:
# Obtener la ruta base (carpeta) de este archivo
ruta_base = os.path.dirname(os.path.abspath('Limpieza Base de datos-dias'))

# Definir el nombre de la carpeta donde se encuentran los archivos CSV
nombre_carpeta = 'Basesdatos'

# Construir las rutas completas de los archivos CSV dentro de la carpeta
ruta_contaminantes = os.path.join(ruta_base, nombre_carpeta, 'codigos contaminantes.csv')
ruta_estaciones = os.path.join(ruta_base, nombre_carpeta, 'Estacionesfinal.csv')
ruta_final = os.path.join(ruta_base, nombre_carpeta, 'final.csv')

# Cargar los archivos CSV en DataFrames
contaminantes = pd.read_csv(ruta_contaminantes)
estaciones = pd.read_csv(ruta_estaciones, sep=';')

In [ ]:
# Visualizamos tabla contaminantes
contaminantes

Codi_Contaminant                Desc_Contaminant Unitats
0                  1                             SO2   µg/m³
1                  6                              CO   mg/m³
2                  7                              NO   µg/m³
3                  8                             NO2   µg/m³
4                  9                           PM2.5   µg/m³
5                 10                            PM10   µg/m³
6                 12                             NOx   µg/m³
7                 14                              O3   µg/m³
8                 22                    Black Carbon   µg/m³
9                101                            SO2*   µg/m³
10               106                             CO*   mg/m³
11               107                             NO*   µg/m³
12               108                            NO2*   µg/m³
13               109                          PM2.5*   µg/m³
14               110                           PM10*   µg/m³
15               112                            Nox*   µg/m³
16               114                             O3*   µg/m³
17               996   Flow 2 (Mesura interna equip)     NaN
18               997   Flow 1 (Mesura interna equip)     NaN
19               998  Flow C (Mesura interna equip)         
20               999           Biomassa Black Carbon       %

In [ ]:
# Eliminamos valores NaN y visualizamos
estaciones = estaciones.dropna()

# Ver DataFrame resultante
estaciones

Estacio  Longitud    Latitud  Codi_districte        Nom_districte  \
0       4.0    2,2045    41,4039            10.0           Sant Marti   
1      42.0    2,1331    41,3788             3.0       Sants-Montjuic   
2      43.0    2,1538    41,3853             5.0             Eixample   
3      44.0    2,1534    41,3987             6.0               Gracia   
4      50.0    2,1874    41,3864             1.0         Ciutat Vella   
5      54.0     2,148    41,4261             7.0       Horta-Guinardo   
6      57.0    2,1151    41,3875             4.0            Les Corts   
7      58.0    2,1239   41,41843             5.0  Sarria-Sant Gervasi   
8      59.0  2,186138  41,421039             7.0       Horta-Guinardo   
9      60.0  2,183060   41,44767             8.0      Latrinitat Nova   
10     61.0   2,14073   41,35099             3.0  La Marina del Prat    

    Codi_barri                              Nom_barri  
0         68.0                            el Poblenou  
1         18.0                                  Sants  
2          9.0         la Nova Esquerra de l'Eixample  
3         31.0                      la Vila de Gracia  
4          4.0  Sant Pere, Santa Caterina i la Ribera  
5         41.0                       la Vall d'Hebron  
6         21.0                              Pedralbes  
7         22.0     Vallvidrera-el Tibidabo-les Planes  
8         35.0                            El Guinardó  
9         53.0                            Nou Barris   
10        12.0                         Sants-Montjuic

In [ ]:

# Acceder donde se encuentran los archivos CSV
folder_path = 'datos por meses'

# Obtener la lista de nombres de archivos en el directorio
file_list = os.listdir(folder_path)

# Filtrar los archivos que comienzan con 2020, 2021, 2022, 2023 o el año 2019 seguido del mes y tienen extensión CSV
filtered_files = [
    file for file in file_list
    if file.startswith(('2020', '2021', '2022','2023', '2019_04',
                        '2019_05', '2019_06', '2019_07', '2019_08',
                        '2019_09', '2019_10', '2019_11', '2019_12')) and file.endswith('.csv')
]

# Crear una lista para almacenar los DataFrames de los archivos CSV
data_frames = []

# Leer cada archivo y cargarlo como DataFrame de forma iterativa
for file in filtered_files:
    file_path = os.path.join(folder_path, file)
    if file.startswith(('2019_06', '2019_07')):
        df = pd.read_csv(file_path, sep=';')
    else:
        df = pd.read_csv(file_path, sep=',')
    data_frames.append(df)

# Concatenar los DataFrames en uno solo
combined_df = pd.concat(data_frames, ignore_index=True)

# Ahora puedes trabajar con el DataFrame combinado
print(combined_df.shape)

(81880, 57)


In [ ]:
# Eliminamos columnas innecesarias
combined_df = combined_df.drop(columns=['CODI_PROVINCIA','PROVINCIA','CODI_MUNICIPI', 'MUNICIPI'])

# Expresión regular para seleccionar columnas que empiezan por "V"
pattern = r'^V'

# Filtrar las columnas que cumplen con el patrón
filtered_columns = combined_df.filter(regex=pattern)

# Eliminar las columnas del DataFrame
combined_df = combined_df.drop(columns=filtered_columns)

# Igualamos columna H24 a H23
combined_df['H24'] = combined_df['H23']

# Ver el DataFrame resultante
print(combined_df.head())

   ESTACIO  CODI_CONTAMINANT   ANY  MES  DIA  H01  H02  H03  H04  H05  ...  \
0        4                 7  2019    4    2  1.0  1.0  3.0  1.0  1.0  ...   
1        4                 7  2019    4    3  1.0  1.0  1.0  1.0  1.0  ...   
2        4                 7  2019    4    4  1.0  1.0  2.0  4.0  1.0  ...   
3        4                 7  2019    4    5  2.0  1.0  8.0  2.0  2.0  ...   
4        4                 7  2019    4    6  1.0  1.0  1.0  1.0  1.0  ...   

   H15  H16  H17  H18  H19  H20  H21  H22  H23  H24  
0  NaN  NaN  8.0  7.0  7.0  8.0  3.0  1.0  1.0  1.0  
1  6.0  3.0  6.0  3.0  3.0  3.0  2.0  2.0  2.0  2.0  
2  8.0  5.0  4.0  4.0  3.0  2.0  2.0  1.0  2.0  2.0  
3  4.0  5.0  5.0  5.0  3.0  3.0  2.0  2.0  1.0  1.0  
4  3.0  3.0  5.0  4.0  4.0  6.0  2.0  2.0  1.0  1.0  

[5 rows x 29 columns]


In [ ]:
# Filtrar las observaciones con los valores deseados en la columna 'CODI_CONTAMINANT'
filtered_df = combined_df[combined_df['CODI_CONTAMINANT'].isin([8, 10, 12, 7])]

# Verificar la forma del DataFrame filtrado
print(filtered_df.shape)

(45140, 29)


In [ ]:
# Ver el DataFrame resultante
filtered_df.head()

ESTACIO  CODI_CONTAMINANT   ANY  MES  DIA  H01  H02  H03  H04  H05  ...  \
0        4                 7  2019    4    2  1.0  1.0  3.0  1.0  1.0  ...   
1        4                 7  2019    4    3  1.0  1.0  1.0  1.0  1.0  ...   
2        4                 7  2019    4    4  1.0  1.0  2.0  4.0  1.0  ...   
3        4                 7  2019    4    5  2.0  1.0  8.0  2.0  2.0  ...   
4        4                 7  2019    4    6  1.0  1.0  1.0  1.0  1.0  ...   

   H15  H16  H17  H18  H19  H20  H21  H22  H23  H24  
0  NaN  NaN  8.0  7.0  7.0  8.0  3.0  1.0  1.0  1.0  
1  6.0  3.0  6.0  3.0  3.0  3.0  2.0  2.0  2.0  2.0  
2  8.0  5.0  4.0  4.0  3.0  2.0  2.0  1.0  2.0  2.0  
3  4.0  5.0  5.0  5.0  3.0  3.0  2.0  2.0  1.0  1.0  
4  3.0  3.0  5.0  4.0  4.0  6.0  2.0  2.0  1.0  1.0  

[5 rows x 29 columns]

In [ ]:
# Lista de columnas de horas con nombres como 'H01', 'H02', ..., 'H24'
hour_columns = [f'H{i:02d}' for i in range(1, 25)]

# Derretir el DataFrame para convertir las columnas de horas en una sola columna
melted_df = pd.melt(filtered_df, id_vars=['ESTACIO', 'CODI_CONTAMINANT', 'ANY', 'MES', 'DIA'],
                    value_vars=hour_columns, var_name='HORA', value_name='CONTAMINANTE')

# Eliminar el prefijo 'H' de los valores en la columna 'HORA' y convertirlos a tipo entero
melted_df['HORA'] = melted_df['HORA'].str.replace('H', '').astype(int)

# Eliminar las filas que contienen valores faltantes (NaN)
melted_df = melted_df.dropna()

# Ordenar el DataFrame por 'ESTACIO', 'ANY', 'MES', 'DIA' y 'HORA', y restablecer los índices
melted_df = melted_df.sort_values(['ESTACIO', 'ANY', 'MES', 'DIA', 'HORA']).reset_index(drop=True)

# Mostrar el DataFrame resultante
melted_df.head()

ESTACIO  CODI_CONTAMINANT   ANY  MES  DIA  HORA  CONTAMINANTE
0        4                 7  2019    4    2     1           1.0
1        4                 8  2019    4    2     1          22.0
2        4                10  2019    4    2     1          19.0
3        4                12  2019    4    2     1          22.0
4        4                 7  2019    4    2     2           1.0

In [ ]:
# Realizar un pivote del DataFrame melted_df para obtener un formato de tabla más conveniente
pivoted_df = melted_df.pivot(index=['ESTACIO', 'ANY', 'MES', 'DIA', 'HORA'],
                             columns='CODI_CONTAMINANT', values='CONTAMINANTE').reset_index()

# Eliminar el nombre de las columnas del índice
pivoted_df.columns.name = None

# Ordenar el DataFrame por 'ESTACIO', 'ANY', 'MES', 'DIA' y 'HORA'
pivoted_df = pivoted_df.sort_values(['ESTACIO', 'ANY', 'MES', 'DIA', 'HORA']).reset_index(drop=True)

In [ ]:
# Crear un diccionario de mapeo para los nuevos nombres de columna
column_mapping = {7: 'NO (µg/m³)',
                  8: 'NO2 (µg/m³)',
                  10: 'PM10 (µg/m³)',
                  12: 'NOx (µg/m³)',
                  'ESTACIO': 'Estación',
                  'ANY': 'Año',
                  'MES': 'Mes',
                  'DIA': 'Dia',
                  'HORA': 'Hora'}

# Renombrar las columnas
pivoted_df = pivoted_df.rename(columns=column_mapping)

In [ ]:
# Obtener las columnas relevantes para el cálculo de similitud
similarity_columns = ['Estación', 'Año', 'Mes', 'Dia', 'Hora']

# Iterar sobre las columnas específicas del DataFrame
columns_to_fill = ['NO (µg/m³)', 'NO2 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)']
for column in columns_to_fill:
    # Verificar si la columna tiene valores faltantes (NaN)
    if pivoted_df[column].isnull().any():
        # Utilizar el método interpolate para rellenar los valores faltantes
        pivoted_df[column] = pivoted_df.groupby(similarity_columns)[column].apply(lambda x: x.interpolate())

In [ ]:
# Eliminar valores NaN
pivoted_df = pivoted_df.dropna()

In [ ]:
# Filtrar observaciones coincidentes de las estaciones 4, 54 y 44
matching_observations = pivoted_df[pivoted_df['Estación'].isin([4, 54, 44])]

# Calcular la media de cada grupo de año, mes, día y hora para las observaciones coincidentes
mean_values = matching_observations.groupby(['Año', 'Mes', 'Dia', 'Hora']).mean()

# Generar un valor aleatorio dentro del rango de ±0.05 para cada media calculada
random_offset = random.uniform(-0.05, 0.05)
mean_values += random_offset

# Crear un DataFrame para las observaciones de la estación 59 para las mismas combinaciones de año, mes, día y hora
station_59_observations = mean_values.reset_index()
station_59_observations['Estación'] = 59

# Concatenar el DataFrame de la estación 59 con el DataFrame original
pivoted_df = pd.concat([pivoted_df, station_59_observations], ignore_index=True)

In [ ]:
# Filtrar observaciones coincidentes de las estaciones 4, 54 y 44
matching_observations = pivoted_df[pivoted_df['Estación'].isin([58])]

# Calcular la media de cada grupo de año, mes, día y hora para las observaciones coincidentes
mean_values = matching_observations.groupby(['Año', 'Mes', 'Dia', 'Hora']).mean()

# Generar un valor aleatorio dentro del rango de ±0.05 para cada media calculada
random_offset = random.uniform(-4, 4)
mean_values += random_offset

# Crear un DataFrame para las observaciones de la estación 59 para las mismas combinaciones de año, mes, día y hora
station_60_observations = mean_values.reset_index()
station_60_observations['Estación'] = 60

# Concatenar el DataFrame de la estación 59 con el DataFrame original
pivoted_df = pd.concat([pivoted_df, station_60_observations], ignore_index=True)


In [ ]:
# Filtrar observaciones coincidentes de las estaciones 4, 54 y 44
matching_observations = pivoted_df[pivoted_df['Estación'].isin([50,43])]

# Calcular la media de cada grupo de año, mes, día y hora para las observaciones coincidentes
mean_values = matching_observations.groupby(['Año', 'Mes', 'Dia', 'Hora']).mean()

# Generar un valor aleatorio dentro del rango de ±0.05 para cada media calculada
random_offset = random.uniform(-0.05, 0.05)
mean_values += random_offset

# Crear un DataFrame para las observaciones de la estación 59 para las mismas combinaciones de año, mes, día y hora
station_61_observations = mean_values.reset_index()
station_61_observations['Estación'] = 61

# Concatenar el DataFrame de la estación 59 con el DataFrame original
pivoted_df = pd.concat([pivoted_df, station_61_observations], ignore_index=True)


In [ ]:
# Redondear las columnas 'NO (µg/m³)', 'NO2 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)' a 0 decimales
pivoted_df[['NO (µg/m³)', 'NO2 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)']] = pivoted_df[['NO (µg/m³)', 'NO2 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)']].round(0)

# Reemplazar todos los valores "-2" por "0"
pivoted_df = pivoted_df.replace(-2, 0)

# Ver DataFrame resultante
pivoted_df.head()

Estación   Año  Mes  Dia  Hora  NO (µg/m³)  NO2 (µg/m³)  PM10 (µg/m³)  \
0         4  2019    4    2     1         1.0         22.0          19.0   
1         4  2019    4    2     2         1.0         37.0          20.0   
2         4  2019    4    2     3         3.0         75.0          27.0   
3         4  2019    4    2     4         1.0         57.0          33.0   
4         4  2019    4    2     5         1.0         45.0          30.0   

   NOx (µg/m³)  
0         22.0  
1         38.0  
2         79.0  
3         59.0  
4         46.0

In [ ]:
# Ver resumen estadístico de las columnas numéricas en el DataFrame pivoted_df
pivoted_df.describe()

Estación            Año            Mes            Dia  \
count  298770.000000  298770.000000  298770.000000  298770.000000   
mean       48.341018    2020.949630       6.395378      15.727787   
std        17.471724       1.230614       3.394272       8.773607   
min         4.000000    2019.000000       1.000000       1.000000   
25%        44.000000    2020.000000       4.000000       8.000000   
50%        57.000000    2021.000000       6.000000      16.000000   
75%        59.000000    2022.000000       9.000000      23.000000   
max        61.000000    2023.000000      12.000000      31.000000   

                Hora     NO (µg/m³)    NO2 (µg/m³)   PM10 (µg/m³)  \
count  298770.000000  298770.000000  298770.000000  298770.000000   
mean       12.497982       9.949680      26.357034      22.713964   
std         6.945284      22.283913      20.246070      17.458673   
min         1.000000       0.000000       1.000000       1.000000   
25%         6.000000       1.000000      10.000000      13.000000   
50%        13.000000       4.000000      21.000000      20.000000   
75%        19.000000       9.000000      37.000000      28.000000   
max        24.000000    1371.000000     233.000000    2297.000000   

         NOx (µg/m³)  
count  298770.000000  
mean       40.621264  
std        49.388898  
min         1.000000  
25%        12.000000  
50%        26.000000  
75%        51.000000  
max      2785.000000

In [ ]:
# Guardar archivo en excel

# pivoted_df.to_excel('base.xlsx')

In [ ]:
# Unir las DataFrames en función de la estación
df_completo = pd.merge(estaciones, pivoted_df, left_on='Estacio', right_on='Estación')

# Eliminar la columna de estación redundante
df_completo = df_completo.drop(columns = ['Estacio','Codi_districte','Nom_districte','Codi_barri','Nom_barri'], axis=1)

# Ver DataFrame resultante
df_completo.head()

Longitud  Latitud  Estación   Año  Mes  Dia  Hora  NO (µg/m³)  NO2 (µg/m³)  \
0   2,2045  41,4039         4  2019    4    2     1         1.0         22.0   
1   2,2045  41,4039         4  2019    4    2     2         1.0         37.0   
2   2,2045  41,4039         4  2019    4    2     3         3.0         75.0   
3   2,2045  41,4039         4  2019    4    2     4         1.0         57.0   
4   2,2045  41,4039         4  2019    4    2     5         1.0         45.0   

   PM10 (µg/m³)  NOx (µg/m³)  
0          19.0         22.0  
1          20.0         38.0  
2          27.0         79.0  
3          33.0         59.0  
4          30.0         46.0

In [ ]:
# Cargar datos Dataframe Final
datos = pd.read_csv(ruta_final)
datos.head()

WKT          Area  \
0  MULTIPOLYGON (((432519.740480454 4584126.94788...  1.104976e+06   
1  MULTIPOLYGON (((427670.087512081 4582617.52040...  1.412573e+06   
2  MULTIPOLYGON (((430774.340173074 4581547.35716...  1.098393e+06   
3  MULTIPOLYGON (((430603.978969729 4583791.30087...  6.495332e+05   
4  MULTIPOLYGON (((431850.501913589 4580899.18102...  8.419051e+05   

         Perim        Coord_X       Coord_Y  Estacio  Longitud  Latitud  \
0  5138.082353  433448.211310  4.584901e+06        4    2.2045  41.4039   
1  4958.808464  427747.944040  4.582278e+06       42    2.1331  41.3788   
2  5557.372878  430732.006258  4.581363e+06       43    2.1538  41.3853   
3  3854.187732  430302.710038  4.584403e+06       44    2.1534  41.3987   
4  5683.004856  431315.817811  4.581595e+06       50    2.1874  41.3864   

   Codi_districte   Nom_districte  Codi_barri  \
0              10      Sant Marti          68   
1               3  Sants-Montjuic          18   
2               5        Eixample           9   
3               6          Gracia          31   
4               1    Ciutat Vella           4   

                               Nom_barri  
0                            el Poblenou  
1                                  Sants  
2         la Nova Esquerra de l'Eixample  
3                      la Vila de Gracia  
4  Sant Pere, Santa Caterina i la Ribera

In [ ]:
# Asignar los valores de la columna 'WKT' del DataFrame 'datos' al DataFrame 'df_completo'
df_completo['WKT'] = df_completo['Estación'].map(datos.set_index('Estacio')['WKT'])

# Eliminar columna WKT.
df_completo = df_completo.drop( columns = ['WKT'], axis = 1)

# Ver DataFrame resultante
df_completo.head()

Longitud  Latitud  Estación   Año  Mes  Dia  Hora  NO (µg/m³)  NO2 (µg/m³)  \
0   2,2045  41,4039         4  2019    4    2     1         1.0         22.0   
1   2,2045  41,4039         4  2019    4    2     2         1.0         37.0   
2   2,2045  41,4039         4  2019    4    2     3         3.0         75.0   
3   2,2045  41,4039         4  2019    4    2     4         1.0         57.0   
4   2,2045  41,4039         4  2019    4    2     5         1.0         45.0   

   PM10 (µg/m³)  NOx (µg/m³)  
0          19.0         22.0  
1          20.0         38.0  
2          27.0         79.0  
3          33.0         59.0  
4          30.0         46.0

In [ ]:
#df_completo.to_excel('df_final_dias.xlsx')

In [ ]:
# Convertir la columna 'WKT' a objetos de geometría
datos['geometry'] = gpd.GeoSeries.from_wkt(datos['WKT'])

# Crear un GeoDataFrame a partir de df_completo
gdf_completo = gpd.GeoDataFrame(datos, geometry='geometry')

In [ ]:
# Crear gráfico de Barcelona separado por distritos
fig, ax = plt.subplots()
gdf_completo.plot(ax=ax)
plt.show()